In [1]:
import os

from kompy import KomootConnector
import os

In [2]:
connector = KomootConnector(
    email=os.environ['KOMOOT_EMAIL'],
    password=os.environ['KOMOOT_PASSWORD'],
)

In [3]:
tours = connector.get_tours()

2023-11-21 14:56:49,597 - KomootConnector - WARNING - No user identifier provided, using the currently logged user: 2453610435162
2023-11-21 14:56:49,597 No user identifier provided, using the currently logged user: 2453610435162
2023-11-21 14:56:49,598 - KomootConnector - WARNING - No sort field provided, using default sort field: date
2023-11-21 14:56:49,598 No sort field provided, using default sort field: date
2023-11-21 14:56:53,435 No vector map image found.
2023-11-21 14:56:53,435 No vector map image found.
2023-11-21 14:56:53,436 No vector map image found.
2023-11-21 14:56:53,436 No vector map image found.
2023-11-21 14:56:53,436 No vector map image found.
2023-11-21 14:56:53,437 No vector map image found.
2023-11-21 14:56:53,437 No vector map image found.
2023-11-21 14:56:53,438 No vector map image found.
2023-11-21 14:56:53,438 No vector map image found.
2023-11-21 14:56:53,439 No vector map image found.
2023-11-21 14:56:53,439 No vector map image found.
2023-11-21 14:56:53,4

In [4]:
tour1 = tours[0]

In [5]:
tour1.generate_gpx_track(authentication=connector.authentication)

In [6]:
gpx_track = connector.get_tour_by_id(tour_identifier='1336116394', object_type='gpx')

In [7]:
for track in gpx_track.tracks:
    print(track)
    for segment in track.segments:
        print(segment)
        for point in segment.points:
            print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevation}')

GPXTrack(name='Vertical Mont Mary 2250', segments=[GPXTrackSegment(points=[...])])
GPXTrackSegment(points=[...])
Point at (45.744129,7.360656) -> 564.987481
Point at (45.744145,7.360608) -> 564.987481
Point at (45.744243,7.359599) -> 564.987481
Point at (45.744532,7.358742) -> 568.930097
Point at (45.744614,7.358681) -> 569.640212
Point at (45.745298,7.358686) -> 574.897471
Point at (45.745686,7.358651) -> 577.885523
Point at (45.745886,7.358468) -> 580.127046
Point at (45.746334,7.358445) -> 585.175914
Point at (45.746819,7.358466) -> 590.640753
Point at (45.747218,7.358431) -> 595.142929
Point at (45.747541,7.358412) -> 598.783802
Point at (45.747991,7.358413) -> 604.056532
Point at (45.747922,7.357806) -> 609.096678
Point at (45.747693,7.357172) -> 614.947326
Point at (45.747796,7.35693) -> 617.270403
Point at (45.747966,7.3568) -> 619.533408
Point at (45.748172,7.356529) -> 624.139156
Point at (45.74823,7.356329) -> 626.807866
Point at (45.748443,7.355846) -> 633.848154
Point at (4

In [8]:
for waypoint in gpx_track.waypoints:
    print(f'waypoint {waypoint.name} -> ({waypoint.latitude},{waypoint.longitude})')

In [9]:
for route in gpx_track.routes:
    print('Route:')
    for point in route.points:
        print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevtion}')


In [10]:
from gpxpy.gpx import GPXTrackPoint
from geopy import distance
import math

def calculate_initial_compass_bearing(first_point: GPXTrackPoint, second_point: GPXTrackPoint):
    """
    Calculates the bearing between two points.

    The formula used to calculate the bearing is:
        θ = atan2(sin(Δlong).cos(lat2), cos(lat1).sin(lat2) - sin(lat1).cos(lat2).cos(Δlong))

    :param first_point: The first point
    :param second_point: The second point
    :return: The bearing between the two points in degrees
    """

    lat1 = math.radians(first_point.latitude)
    lat2 = math.radians(second_point.latitude)
    diffLong = math.radians(second_point.longitude - first_point.longitude)

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Normalize the initial bearing
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing



def process_gpx_for_turns(gpx_data):
    processed_segments = []
    for track in gpx_data.tracks:
        for segment in track.segments:
            # Initialize previous bearing
            previous_bearing = None

            for i in range(len(segment.points)-1):
                point1 = segment.points[i]
                point2 = segment.points[i+1]

                # Calculate the elevation change
                elevation_change = point2.elevation - point1.elevation
                positive_elevation_change = elevation_change if elevation_change > 0 else 0
                negative_elevation_change = elevation_change if elevation_change < 0 else 0
                m_distance = distance.geodesic(
                        (point1.latitude, point1.longitude),
                        (point2.latitude,point2.longitude)
                    ).m
                slope_angle = math.atan(elevation_change/m_distance)
                segment_type = "Uphill" if elevation_change > 0 else "Downhill" if elevation_change < 0 else "Flat"

                # Calculate the bearing
                if previous_bearing is not None:
                    current_bearing = calculate_initial_compass_bearing(
                        first_point=point1,
                        second_point=point2,
                    )
                    bearing_change = current_bearing - previous_bearing

                    # Determine left or right turn
                    if bearing_change > 10:
                        turn_direction = "Right"
                    elif bearing_change < -10:
                        turn_direction = "Left"
                    else:
                        turn_direction = "Straight"

                    # Classify the turn strength
                    if turn_direction != "Straight":
                        if abs(bearing_change) > 45:
                            turn_strength = "Strong"
                        else:
                            turn_strength = "Mild"
                    else:
                        turn_strength = "N/A"

                    previous_bearing = current_bearing
                else:
                    turn_direction = "N/A"
                    turn_strength = "N/A"
                    previous_bearing = calculate_initial_compass_bearing(
                        first_point=point1,
                        second_point=point2,
                    )
                processed_segments.append({
                    'segment': i,
                    'latitude': point1.latitude,
                    'longitude': point1.longitude,
                    'altitude': point1.elevation,
                    'elevation_change': elevation_change,
                    'positive_elevation_change': positive_elevation_change,
                    'negative_elevation_change': negative_elevation_change,
                    'segment_type': segment_type,
                    'turn_direction': turn_direction,
                    'turn_strength': turn_strength,
                    'distance': m_distance,
                    'slope_angle': slope_angle,
                })

    return processed_segments

In [11]:
processed_segments = process_gpx_for_turns(gpx_track)

In [12]:
import pandas as pd
df = pd.DataFrame(processed_segments)
print(df.shape)
display(df.head(30))

(540, 12)


,segment,latitude,longitude,altitude,elevation_change,positive_elevation_change,negative_elevation_change,segment_type,turn_direction,turn_strength,distance,slope_angle
0,0,45.744129,7.360656,564.987481,0.000000,0.000000,0,Flat,N/A,N/A,4.137059,0.000000
1,1,45.744145,7.360608,564.987481,0.000000,0.000000,0,Flat,Left,Mild,79.271767,0.000000
2,2,45.744243,7.359599,564.987481,3.942616,3.942616,0,Uphill,Right,Mild,74.023488,0.053211
3,3,45.744532,7.358742,568.930097,0.710115,0.710115,0,Uphill,Right,Mild,10.276117,0.068994
4,4,45.744614,7.358681,569.640212,5.257259,5.257259,0,Uphill,Left,Strong,76.025089,0.069042
5,5,45.745298,7.358686,574.897471,2.988052,2.988052,0,Uphill,Right,Strong,43.210704,0.069041
6,6,45.745686,7.358651,577.885523,2.241523,2.241523,0,Uphill,Left,Mild,26.399509,0.084705
7,7,45.745886,7.358468,580.127046,5.048868,5.048868,0,Uphill,Right,Mild,49.825724,0.100986
8,8,45.746334,7.358445,585.175914,5.464839,5.464839,0,Uphill,Left,Strong,53.930752,0.100986
9,9,45.746819,7.358466,590.640753,4.502176,4.502176,0,Uphill,Right,Strong,44.430957,0.100985


In [13]:
tour1.generate_coordinates(authentication=connector.authentication)

True

In [17]:
df_agg = df.groupby((df['turn_direction'] != df['turn_direction'].shift()).cumsum()).agg(
    {
        'segment': list,
        'distance':  'sum',
        'latitude': ['first', 'last'],
        'longitude': ['first', 'last'],
        'altitude': ['first', 'last'],
        'elevation_change': list,
        'positive_elevation_change': 'sum',
        'negative_elevation_change': 'sum',
        'segment_type': list,
        'turn_direction': 'first',
        'turn_strength': list,
        'slope_angle': list,
    }
)

In [19]:
print(df_agg.shape)
display(df_agg)

(412, 15)


segment   distance   latitude            longitude  \
                      list        sum      first       last     first   
turn_direction                                                          
1                      [0]   4.137059  45.744129  45.744129  7.360656   
2                      [1]  79.271767  45.744145  45.744145  7.360608   
3                   [2, 3]  84.299605  45.744243  45.744532  7.359599   
4                      [4]  76.025089  45.744614  45.744614  7.358681   
5                      [5]  43.210704  45.745298  45.745298  7.358686   
...                    ...        ...        ...        ...       ...   
408                  [534]   8.111028  45.792858  45.792858  7.373022   
409                  [535]   6.310048  45.792930  45.792930  7.373005   
410             [536, 537]  11.275582  45.792943  45.792972  7.372926   
411                  [538]   4.020143  45.793023  45.793023  7.372866   
412                  [539]   5.290776  45.793059  45.793059  7.372871   

                             altitude               \
                    last        first         last   
turn_direction                                       
1               7.360656   564.987481   564.987481   
2               7.360608   564.987481   564.987481   
3               7.358742   564.987481   568.930097   
4               7.358681   569.640212   569.640212   
5               7.358686   574.897471   574.897471   
...                  ...          ...          ...   
408             7.373022  2711.098181  2711.098181   
409             7.373005  2711.098181  2711.098181   
410             7.372867  2711.098181  2711.098181   
411             7.372866  2711.098181  2711.098181   
412             7.372871  2711.098181  2711.098181   

                                        elevation_change  \
                                                    list   
turn_direction                                             
1                                                  [0.0]   
2                                                  [0.0]   
3               [3.9426160000000436, 0.7101149999999734]   
4                                    [5.257258999999976]   
5                                    [2.988052000000039]   
...                                                  ...   
408                                                [0.0]   
409                                                [0.0]   
410                                           [0.0, 0.0]   
411                                                [0.0]   
412                                                [0.0]   

               positive_elevation_change negative_elevation_change  \
                                     sum                       sum   
turn_direction                                                       
1                               0.000000                         0   
2                               0.000000                         0   
3                               4.652731                         0   
4                               5.257259                         0   
5                               2.988052                         0   
...                                  ...                       ...   
408                             0.000000                         0   
409                             0.000000                         0   
410                             0.000000                         0   
411                             0.000000                         0   
412                             0.000000                         0   

                    segment_type turn_direction   turn_strength  \
                            list          first            list   
turn_direction                                                    
1                         [Flat]            N/A           [N/A]   
2                         [Flat]           Left          [Mild]   
3               [Uphill, Uphill]          Right    [Mild, Mild]   
4              

In [ ]:
def significant_elevation_change(elevation_change):
    


def significant_turn(bearing_change):
    pass


def split_to_segments(gpx_data):
    segments = []
    current_segment = {'start_point': 0, 'elevation': 'flat', 'turn': 'straight'}
    for track in gpx_data.tracks:
        for segment in track.segments:
            for i in range(len(segment.points)-1):
                point1 = segment.points[i]
                point2 = segment.points[i+1]
                elevation_change = point2.elevation - point1.elevation
                bearing_change = calculate_initial_compass_bearing(
                    first_point=point1,
                    second_point=point2,
                )
                
                if significant_elevation_change(elevation_change) or significant_turn(bearing_change):
                    # Finish the current segment
                    current_segment['end_point'] = i
                    segments.append(current_segment)

                    # Start a new segment
                    current_segment = {'start_point': i, 'elevation': determine_elevation_type(elevation_change), 'turn': determine_turn_direction(bearing_change)}

    

In [ ]:
segments = []
current_segment = {'start_point': 0, 'elevation': 'flat', 'turn': 'straight'}

for i in range(1, len(gpx_points)):
    elevation_change = gpx_points[i].elevation - gpx_points[i-1].elevation
    bearing_change = calculate_bearing_change(gpx_points[i-1], gpx_points[i])

    if significant_elevation_change(elevation_change) or significant_turn(bearing_change):
        # Finish the current segment
        current_segment['end_point'] = i
        segments.append(current_segment)

        # Start a new segment
        current_segment = {'start_point': i, 'elevation': determine_elevation_type(elevation_change), 'turn': determine_turn_direction(bearing_change)}
    # Otherwise, continue accumulating the current segment

# Add the final segment
current_segment['end_point'] = len(gpx_points) - 1
segments.append(current_segment)
